 ## <b>Connecting vxi11 compatible devices via Ethernet </b>

In [ ]:
!pip3 install python-vxi11

In [ ]:
import vxi11
import time

In [ ]:
keysight_ip = '192.168.0.8'

In [ ]:
Keysight = vxi11.Instrument(keysight_ip)
print(Keysight.ask("*IDN?"))

In [ ]:
rigol_ip = '192.168.0.14'

In [ ]:
# Rigol = vxi11.Instrument(rigol_ip)
# print(Rigol.ask("*IDN?"))

## <b> Interacting with a Brüel & Kjær Type 3050-B-6 using Python </b>

In [ ]:
cd ../PyBnK

In [ ]:
from bnk.bnk import WavHeader, OpenWav, Instrument

In [ ]:
bnk_ip = '192.168.0.9'
ADAC = Instrument(bnk_ip)
print(ADAC) # Show some info about the BnK device

## <b> Python Wrapper for Keysight 33500B - Modified from Carlin's code</b>

In [ ]:
class keysight():
    
    def __init__(self,instrument_ID=keysight_ip):
        self.Keysight =  vxi11.Instrument(instrument_ID)
        print(self.Keysight.ask("*IDN?"))
        
    def enable(self):
        self.Keysight.write("OUTPut ON")
        
    def disable(self):
        self.Keysight.write("OUTPut OFF") 
        
    def enable_high_impedance(self):
        self.Keysight.write("OUTPut:LOAD INF")  
        
    def sine(self,frequency=500,amplitude=0.01,offset=0):
        self.Keysight.write(f"VOLTage:OFFSet {offset}")
        self.Keysight.write("VOLTage:UNIT VRMS")
        self.Keysight.write(f"VOLTage {amplitude}")
        self.Keysight.write("FUNCtion SINusoid")
        self.Keysight.write(f"FREQuency {frequency}")
        
    def whitenoise(self, bandwidth=1500, amplitude=0.01):
        self.Keysight.write("VOLTage:UNIT VRMS")
        self.Keysight.write(f"VOLTage {amplitude}")
        self.Keysight.write("FUNCtion NOISe")
        self.Keysight.write(f"FUNCtion:NOISe:BANDwidth {bandwidth}")
        
    def enable_burst(self,frequency=2000,ncycles=1,int_period=2,amplitude=0.01):
        self.Keysight.write("FUNCtion SINusoid")
        self.Keysight.write(f"FREQuency {frequency}")
        self.Keysight.write(f"VOLTage {amplitude}")
        self.Keysight.write("BURSt:MODE TRIG")
        self.Keysight.write(f"BURSt:NCYCles {ncycles}")
        self.Keysight.write(f"BURSt:INTernal:PERiod {int_period}")
        self.Keysight.write("TRIGger:SOURce IMMediate")
        self.Keysight.write("BURSt:STATe ON")
        
        self.Keysight.write("OUTPut ON")
        
    def disable_burst(self):
        self.Keysight.write("BURSt:STATe OFF")
        self.Keysight.write("OUTPut OFF")
        
    def enable_sweep(self, mode="logarithmic", runTime=10):
        self.Keysight.write("FUNCtion SINusoid")
        self.Keysight.write("SWEep:STATe ON")
        self.Keysight.write(f"SWEep:SPAcing {mode}")
        self.Keysight.write(f"SWEep:TIME {runTime}")
        self.Keysight.write(f"FREQuency:STARt {500}")
        self.Keysight.write(f"FREQuency:STOP {3000}")
        self.Keysight.write("OUTPut ON")
        
    def disable_sweep(self):
        self.Keysight.write("SWEep:STATe OFF")
        self.Keysight.write("OUTPut OFF")

## <b> Connecting to RPI </b>

In [ ]:
#Connecting to the RPi over Wi-Fi
import datetime
import paramiko
from paramiko import SSHClient
from scp import SCPClient
RPi = '192.168.0.7' # This is the IP address when RPi is connected to SHL network, note that this changes depending on different networks unless static IP is set
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.load_system_host_keys()
ssh.connect(RPi,username='pi',password='VM3000Pi4')

In [ ]:
scp = SCPClient(ssh.get_transport())

In [3]:
# File directory for all audio .wav test files
import os

# Change windows directory to Audio Files
os.chdir("C:\\Users\\ricky\\Documents\\VM3000-Microphones\\Audio Files")

# Checking Current File directory
print(os.getcwd())

C:\Users\ricky\Documents\VM3000-Microphones\Audio Files


In [ ]:
# Checking RPI Zero W files using ls

stdin, stdout, stderr = ssh.exec_command('ls -l')
for x in stdout: # stdout is a file, therefore to see the ls contents of the RPI, we use a for loop and print out all lines in stdout
    print(x)

In [ ]:
# client = SSHClient()
# client.load_system_host_keys()
# client.connect('192.168.0.6',username='pi')
#stdin, stdout, stderr = client.exec_command('touch test_file.txt')
#print(stdin)

In [ ]:
# cmd = client.exec_command('arecord -D plughw:1 -c2 -r 48000 -f S16_LE -t wav -V stereo -v -d 50 tester2.wav')

## <b> Class for Test Functions </b>

In [ ]:
class signalTests():    
    def sweepTest(self, mode="logarithmic", recordTime=15, runTime=10, sleepTime=5):
        from paramiko import SSHClient
        from scp import SCPClient
        import datetime
        import time
        
        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        sample_rate = 96000

        cmd = f'cd ~/AudioFiles; arecord -D plughw:1 -c2 -r {sample_rate} -f S16_LE -t wav -V stereo -v -d {recordTime} Pi4sweepTest_{now}.wav'

        print(cmd)
        ssh.exec_command(cmd)
        
        time.sleep(sleepTime) # Buffer time prior to enabling signal generator
        
        testName = "Sweep Test"
        
        sg.enable_sweep(mode, runTime)
        sg.enable()
        time.sleep(runTime)
        sg.disable()
        time.sleep(sleepTime)
        sg.disable_sweep()
        
        print(f"Finished {testName} Output")
        
        while (int(ssh.exec_command('ps -C arecord | wc -l')[1].read()) > 1): # Checking for arecord to finish writing .wav file prior to pulling file using scp
            time.sleep(1)
        
        scp.get(f'~/AudioFiles/Pi4sweepTest_{now}.wav') # Command pulls file into .../Audio Files folder
        
    def sineTest(self, frequency=2000, amplitude=0.01, offset=0, recordTime=15, runTime=10, sleepTime=5):
        from paramiko import SSHClient
        from scp import SCPClient
        import datetime
        import time

        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        sample_rate = 96000

        cmd = f'cd ~/AudioFiles; arecord -D plughw:1 -c2 -r {sample_rate} -f S16_LE -t wav -V stereo -v -d {recordTime} Pi4sineTest_{now}.wav'

        print(cmd)
        ssh.exec_command(cmd)
        
        time.sleep(2.5)
        
        testName = "Sine Wave Test"
        
        sg.sine(frequency,amplitude,offset)
        sg.enable()
        time.sleep(runTime)
        sg.disable()
        time.sleep(sleepTime)

        print(f"Finished {testName} Output")
        
        while (int(ssh.exec_command('ps -C arecord | wc -l')[1].read()) > 1):
            time.sleep(1)
        
        scp.get(f'~/AudioFiles/Pi4sineTest_{now}.wav')
        
    def sineSweepBurstTest(self, frequency=500, amplitude=0.01, offset=0, step=125, recordTime=15, runTime=10, sleepTime=5):
        from paramiko import SSHClient
        from scp import SCPClient
        import datetime
        import time

        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        sample_rate = 96000

        cmd = f'cd ~/AudioFiles; arecord -D plughw:1 -c2 -r {sample_rate} -f S16_LE -t wav -V stereo -v -d {recordTime} Pi4sineSweepBurstTest_{now}.wav'

        print(cmd)
        ssh.exec_command(cmd)
        
        time.sleep(2.5)
        
        testName = "Sine Sweep Burst Test"
        
        sg.sine(frequency,amplitude,offset)
        sg.enable()
        
        for i in range(1,21,1):
            if i < 2:
                frequency=frequency
                sg.sine(frequency,amplitude,offset)
                sg.enable()
                time.sleep(0.2)
                sg.disable()
                time.sleep(0.2)
            else:  
                frequency=frequency+step
                sg.sine(frequency,amplitude,offset)
                sg.enable()
                time.sleep(0.2)
                sg.disable()
                time.sleep(0.2)
            
        time.sleep(sleepTime)    
            
        print(f"Finished {testName} Output")
        
        while (int(ssh.exec_command('ps -C arecord | wc -l')[1].read()) > 1):
            time.sleep(1)
        
        scp.get(f'~/AudioFiles/Pi4sineSweepBurstTest_{now}.wav')        
        
    def whiteNoiseTest(self, bandwidth=500, amplitude=0.01, recordTime=15, runTime=10, sleepTime=2):
        from paramiko import SSHClient
        from scp import SCPClient
        import datetime
        import time

        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        sample_rate = 96000

        cmd = f'cd ~/AudioFiles; arecord -D plughw:1 -c2 -r {sample_rate} -f S16_LE -t wav -V stereo -v -d {recordTime} Pi4whiteNoiseTest_{now}.wav'

        print(cmd)
        ssh.exec_command(cmd)      
        
        time.sleep(2.5)
        
        testName = "White Noise Test"
        
        sg.whitenoise(bandwidth,amplitude)
        sg.enable()
        time.sleep(runTime)
        sg.disable()
        time.sleep(sleepTime)

        print(f"Finished {testName} Output")
        
        while (int(ssh.exec_command('ps -C arecord | wc -l')[1].read()) > 1):
            time.sleep(1)
        
        scp.get(f'~/AudioFiles/Pi4whiteNoiseTest_{now}.wav')
        
    def burstTest(self, frequency=2000, amplitude=0.01, offset=0, ncycles=10, interval=1):
        from paramiko import SSHClient
        from scp import SCPClient
        import datetime
        import time

        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        sample_rate = 96000

        cmd = f'cd ~/AudioFiles; arecord -D plughw:1 -c2 -r {sample_rate} -f S16_LE -t wav -V stereo -v -d {runTime} Pi4burstTest_{now}.wav'

        print(cmd)
        ssh.exec_command(cmd)     
        
        time.sleep(2.5)
        
        testName = "Burst Test"

        t_end = time.time() + 10
        
        while time.time() < t_end:
            sg.enable_burst(frequency,ncycles,interval)
        
        sg.disable_burst()

        print(f"Finished {testName} Output")  
        
        while (int(ssh.exec_command('ps -C arecord | wc -l')[1].read()) > 1):
            time.sleep(1)
        
        scp.get(f'~/AudioFiles/Pi4burstTest_{now}.wav')
        
    def ambientTest(self):
        from paramiko import SSHClient
        from scp import SCPClient
        import datetime
        import time

        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        sample_rate = 96000

        cmd = f'cd ~/AudioFiles; arecord -D plughw:1 -c2 -r {sample_rate} -f S16_LE -t wav -V stereo -v -d {runTime} Pi4ambientTest_{now}.wav'

        print(cmd)
        ssh.exec_command(cmd)   
        
        time.sleep(2.5)
        
        testName = "Ambient Test"
        
        print(f"Finished {testName} Output")
        
        while (int(ssh.exec_command('ps -C arecord | wc -l')[1].read()) > 1):
            time.sleep(1)
        
        scp.get(f'~/AudioFiles/Pi4ambientTest_{now}.wav')

## <b> Test Cases </b>

In [ ]:
sg=keysight()
sg.enable_high_impedance()

In [ ]:
import paramiko
import time
import datetime

In [ ]:
# Sweep Test

test1_name = "Sweep Test"
mode = "logarithmic"
sweep_period = 10

sg.enable_sweep(mode,sweep_period)
sg.enable()
time.sleep(sweep_period)
sg.disable()
time.sleep(sweep_period)
sg.disable_sweep()

print(f"Finished {test1_name} Output")

In [ ]:
# Continuous Sine Wave Test

test2_name = "Sine Wave Test"
frequency = 2000
amplitude = 0.01 # Vrms
offset = 0
sleep_time = 2

sg.sine(frequency,amplitude,offset)
sg.enable()
time.sleep(sleep_time)
sg.disable()
time.sleep(sleep_time)

print(f"Finished {test2_name} Output")

In [ ]:
# White Noise Test

test3_name = "White Noise Test"
bandwidth = 500
amplitude = 0.01 # Vrms
sleep_time = 2

sg.whitenoise(bandwidth,amplitude)
sg.enable()
time.sleep(sleep_time)
sg.disable()s
time.sleep(sleep_time)

print(f"Finished {test3_name} Output")

In [ ]:
# Burst Sweep Test

test4_name = "Burst Sweep Test"
frequency = 2000
ncycles = 10
int_period = 1
sleep_time = 2

sg.enable_burst(frequency,ncycles,int_period)
  
for f in range(2000,5000,1000):
    print(f'Pulsing at {f} Hz')
    Keysight.write(f"FREQuency {f}")
    sg.enable()
    time.sleep(sleep_time)
    sg.disable()
    time.sleep(sleep_time)

sg.disable_burst()
    
print(f"Finished {test4_name} Output")

In [ ]:
# Burst Test Constant Frequency

test5_name = "Constant Frequency Burst Test"
frequency = 2000
ncycles = 10
int_period = 1
sleep_time = 2

sg.enable_burst(frequency,ncycles,int_period)

for i in range(1,11,1):
    print(f'Pulse number {i}')
    Keysight.write(f"FREQuency {frequency}")
    sg.enable()
    time.sleep(sleep_time)
    sg.disable()
    time.sleep(sleep_time)
          
print(f"Finished {test5_name} Output")

## <b> Frequency Response Characteristics Testing Script </b>

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

from scipy.io import wavfile
from scipy.fftpack import fft,fftfreq

import IPython
import numpy as np
import soundfile
import wave

import datetime
import time

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Legend


import IPython.display as ipd

import ffmpeg

In [ ]:
st = signalTests()

#sg=keysight()
#sg.enable_high_impedance()

In [ ]:
#st = signalTests()

### Frequency Sweep Test

In [ ]:
# Running Frequency Sweep Test & recording audio from RPI Zero W 

mode="logarithmic"
runTime=5
sleepTime=2
recordTime=10 # Always specify a recordTime greater than runTime to build in a buffer at the end of audio recording

now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

st.sweepTest(mode,recordTime,runTime,sleepTime)

In [ ]:
# Trimming the empty buffer time of wav file

audio_input = ffmpeg.input(f'Pi4sweepTest_{now}.wav')
#audio_input = ffmpeg.input('sweepTest_20210312_134055.wav')
audio_cut = audio_input.audio.filter('atrim', start=2, end=8.5)
audio_output = ffmpeg.output(audio_cut, f'Pi4sweepTest_{now}_trimmed.wav')
#audio_output = ffmpeg.output(audio_cut, 'sweepTest_20210312_134055_trimmed.wav')
ffmpeg.run(audio_output)

In [ ]:
data, samplerate = soundfile.read(f"Pi4sweepTest_{now}_trimmed.wav")
#data, samplerate = soundfile.read("sweepTest_20210319_154909_trimmed.wav")

In [ ]:
# Play recorded audio
ipd.Audio(f"Pi4sweepTest_{now}_trimmed.wav") # load recent local sweep test file
#ipd.Audio("Pi4sweepTest_20210401_095953_trimmed.wav")

In [ ]:
#recordTime = 15
#samplerate = 96000

# Get x co-ordinates
time=np.arange(0, runTime, 1/samplerate)

# Get y co-ordinates
dataL = data[:,0]
dataR = data[:,1]

# Set up figures
plot1 = figure(
    sizing_mode="stretch_width",
    title = "Frequency Sweep Test Stereo WAV", 
    x_axis_label = 'Time (s)', 
    y_axis_label = 'Normalised Voltage',
    tools = 'xwheel_zoom,xpan,box_zoom,xwheel_pan,save,reset,hover'
    )

# Add a line renderer with legend and line thickness
plot1.line(time, dataL, legend_label="Left Channel Data", line_width=1, line_color="blue")
plot1.line(time, dataR, legend_label="Right Channel Data", line_width=1, line_color="red")

# Makes legends toggleable
plot1.legend.click_policy="hide"

# show the results
output_notebook()
show(plot1)

In [ ]:
samples = data.shape[0]
datafftL = fft(dataL)
datafftR = fft(dataR)
fftabsL = abs(datafftL[1:len(dataL)//2])
fftabsR = abs(datafftR[1:len(dataR)//2])
freqs = fftfreq(samples,1/(samplerate/2))

# Create a new plot
plot1 = figure(
    sizing_mode="stretch_width",
    title = "Frequency Sweep Test FFT", 
    x_axis_type="log",
    y_axis_type="linear",
    x_axis_label = 'Frequency (Hz)', 
    #y_axis_label = 'Normalised Voltage',
    tools = 'xwheel_zoom,xpan,box_zoom,xwheel_pan,save,reset,hover'
    )

# Add a line renderer with legend and line thickness
plot1.line(freqs[:int(freqs.size/2)], fftabsL, legend_label="Left Channel Data FFT", line_width=1, line_color="blue")
plot1.line(freqs[:int(freqs.size/2)], fftabsR, legend_label="Right Channel Data FFT", line_width=1, line_color="red")

plot1.legend.click_policy="hide"

# show the results
output_notebook()
show(plot1)

In [ ]:
#recordTime=15

plt.plot(data[:recordTime*samplerate]) # Plotting stereo audio .wav file
plt.title("Sweep Test plot (Stereo)")

In [ ]:
samples = data.shape[0]
data = data[:,0]

plt.plot(data[:recordTime*samplerate]) # Plotting mono audio .wav file
plt.title("Sweep Test plot (Mono)")

In [ ]:
datafft = fft(data)
fftabs = abs(datafft)
freqs = fftfreq(samples,1/samplerate)

plt.xlim([10, samplerate/2])
plt.xscale( 'log' )
plt.grid( True )
plt.xlabel( 'Frequency (Hz)' )
plt.plot(freqs[:int(freqs.size/2)],fftabs[:int(freqs.size/2)]) # Plotting FFT output of mono wav file
plt.title("Sweep Test FFT Output plot (Mono)")

### Sine Sweep Burst Test

In [ ]:
# Running Sine Wave Test & recording audio from RPI

frequency=500
amplitude=0.03 # Vrms
offset=0
step=125
runTime=8
sleepTime=2
recordTime=12

now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

st.sineSweepBurstTest(frequency,amplitude,offset,step,recordTime,runTime,sleepTime)

In [ ]:
# Trimming the empty buffer time of wav file
audio_input = ffmpeg.input(f'Pi4sineSweepBurstTest_{now}.wav')
#audio_input = ffmpeg.input('sineTest_20210312_134022.wav')
audio_cut = audio_input.audio.filter('atrim', start=2.5, end=12)
audio_output = ffmpeg.output(audio_cut, f'Pi4sineSweepBurstTest_{now}_trimmed.wav')
#audio_output = ffmpeg.output(audio_cut, 'sineTest_20210312_134022_trimmed.wav')
ffmpeg.run(audio_output)

In [ ]:
# Play recorded audio
ipd.Audio(f"Pi4sineSweepBurstTest_{now}_trimmed.wav") # load recent local sine test file
#ipd.Audio("Pi4sineTest_20210401_101202_trimmed.wav")

### Sine Wave Test

In [ ]:
# Running Sine Wave Test & recording audio from RPI Zero W 

frequency=2000
amplitude=0.03 # Vrms
offset=0
runTime=10
sleepTime=2
recordTime=15

now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

st.sineTest(frequency,amplitude,offset,recordTime,runTime,sleepTime)

In [ ]:
# Trimming the empty buffer time of wav file
audio_input = ffmpeg.input(f'Pi4sineTest_{now}.wav')
#audio_input = ffmpeg.input('sineTest_20210312_134022.wav')
audio_cut = audio_input.audio.filter('atrim', start=7, end=9)
audio_output = ffmpeg.output(audio_cut, f'Pi4sineTest_{now}_trimmed.wav')
#audio_output = ffmpeg.output(audio_cut, 'sineTest_20210312_134022_trimmed.wav')
ffmpeg.run(audio_output)

In [ ]:
#data, samplerate = soundfile.read(f"Pi4sineTest_{now}_trimmed.wav")
data, samplerate = soundfile.read("sineTest_20210319_155301_trimmed.wav")

In [ ]:
# Play recorded audio
#ipd.Audio(f"Pi4sineTest_{now}_trimmed.wav") # load recent local sine test file
ipd.Audio("Pi4sineTest_20210401_101202_trimmed.wav")

In [ ]:
recordTime = 15
samplerate = 48000
runTime = 10
time=np.arange(0, runTime, 1/samplerate)

dataL = data[:,0]
dataR = data[:,1]

# Set up figures
plot1 = figure(
    sizing_mode="stretch_width",
    title = "Sine Wave Test Stereo WAV", 
    x_axis_label = 'Time (s)', 
    y_axis_label = 'Normalised Voltage',
    tools = 'xwheel_zoom,xpan,box_zoom,xwheel_pan,save,reset,hover'
    )

# Add a line renderer with legend and line thickness
plot1.line(time, dataL, legend_label="Left Channel Data", line_width=1, line_color="blue")
plot1.line(time, dataR, legend_label="Right Channel Data", line_width=1, line_color="red")

# Makes legends toggleable
plot1.legend.click_policy="hide"

# show the results
output_notebook()
show(plot1)

In [ ]:
samples = data.shape[0]
datafftL = fft(dataL)
datafftR = fft(dataR)
fftabsL = abs(datafftL[1:len(dataL)//2])
fftabsR = abs(datafftR[1:len(dataR)//2])
freqs = fftfreq(samples,1/(samplerate/2))


# Set up figures
plot1 = figure(
    sizing_mode="stretch_width",
    title = "Sine Wave Test FFT", 
    x_axis_type="log",
    y_axis_type="linear",
    x_axis_label = 'Frequency (Hz)', 
    y_axis_label = 'Normalised Voltage',
    tools = 'xwheel_zoom,xpan,box_zoom,xwheel_pan,save,reset,hover'
    )

# Add a line renderer with legend and line thickness
plot1.line(freqs[:int(freqs.size/2)], fftabsL, legend_label="Left Channel Data FFT", line_width=1, line_color="blue")
plot1.line(freqs[:int(freqs.size/2)], fftabsR, legend_label="Right Channel Data FFT", line_width=1, line_color="red")

plot1.legend.click_policy="hide"

# show the results
output_notebook()
show(plot1)

In [ ]:
recordTime=15
frequency=2000

plt.plot(data[:recordTime*samplerate]) # Plotting stereo audio
plt.title(f"Sine Wave Test plot at {frequency} Hz (Stereo)")

In [ ]:
samples = data.shape[0]
data = data[:,0]

plt.plot(data[:recordTime*samplerate]) # Plotting mono audio
plt.title(f"Sine Wave Test plot at {frequency} Hz (Mono)")

In [ ]:
datafft = fft(data)
fftabs = abs(datafft)
freqs = fftfreq(samples,1/samplerate)
plt.xlim([10, samplerate/2])
plt.xscale( 'log' )
plt.grid( True )
plt.xlabel( 'Frequency (Hz)' )
plt.plot(freqs[:int(freqs.size/2)],fftabs[:int(freqs.size/2)])
plt.title(f"Sine Wave Test FFT Output plot at {frequency} Hz")

### White Noise Test

In [ ]:
# Running White Noise Test & recording audio from RPI Zero W

bandwidth=10000
amplitude=0.01 # Vrms
runTime=10
sleepTime=2
recordTime=15

now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

st.whiteNoiseTest(bandwidth,amplitude,recordTime,runTime,sleepTime)

In [ ]:
# Trimming the empty buffer time of wav file
audio_input = ffmpeg.input(f'Pi4whiteNoiseTest_{now}.wav')
#audio_input = ffmpeg.input('whiteNoiseTest_20210312_134138.wav')
audio_cut = audio_input.audio.filter('atrim', start=7, end=9)
audio_output = ffmpeg.output(audio_cut, f'Pi4whiteNoiseTest_{now}_trimmed.wav')
#audio_output = ffmpeg.output(audio_cut, 'whiteNoiseTest_20210312_134138_trimmed.wav')
ffmpeg.run(audio_output)

In [ ]:
data, samplerate = soundfile.read(f"Pi4whiteNoiseTest_{now}_trimmed.wav")
#data, samplerate = soundfile.read("Pi4whiteNoiseTest_20210401_102113_trimmed.wav")

In [ ]:
# Play recorded audio
ipd.Audio(f"Pi4whiteNoiseTest_{now}_trimmed.wav") # load recent local sine test file
#ipd.Audio("Pi4whiteNoiseTest_20210401_102113_trimmed.wav")

In [ ]:
#recordTime = 15
#samplerate = 48000
time=np.arange(0, runTime, 1/samplerate)

dataL = data[:,0]
dataR = data[:,1]

# Set up figures
plot1 = figure(
    sizing_mode="stretch_width",
    title = "White Noise Test Stereo WAV", 
    x_axis_label = 'Time (s)', 
    y_axis_label = 'Normalised Voltage',
    tools = 'xwheel_zoom,xpan,box_zoom,xwheel_pan,save,reset,hover'
    )

# Add a line renderer with legend and line thickness
plot1.line(time, dataL, legend_label="Left Channel Data", line_width=1, line_color="blue")
plot1.line(time, dataR, legend_label="Right Channel Data", line_width=1, line_color="red")

# Makes legends toggleable
plot1.legend.click_policy="hide"

# show the results
output_notebook()
show(plot1)

In [ ]:
samples = data.shape[0]
datafftL = fft(dataL)
datafftR = fft(dataR)
fftabsL = abs(datafftL[1:len(dataL)//2])
fftabsR = abs(datafftR[1:len(dataR)//2])
freqs = fftfreq(samples,1/(samplerate/2))


# Set up figures
plot1 = figure(
    sizing_mode="stretch_width",
    title = "White Noise Test FFT", 
    x_axis_type="linear",
    y_axis_type="log",
    x_axis_label = 'Frequency (Hz)', 
    y_axis_label = 'Normalised Voltage',
    tools = 'xwheel_zoom,xpan,box_zoom,xwheel_pan,save,reset,hover'
    )

# Add a line renderer with legend and line thickness
plot1.line(freqs[:int(freqs.size/2)], fftabsL, legend_label="Left Channel Data FFT", line_width=1, line_color="blue")
plot1.line(freqs[:int(freqs.size/2)], fftabsR, legend_label="Right Channel Data FFT", line_width=1, line_color="red")

plot1.legend.click_policy="hide"

# show the results
output_notebook()
show(plot1)

In [ ]:
plt.plot(data[:recordTime*samplerate]) # Plotting stereo audio
plt.title("White Noise Test plot (Stereo)")

In [ ]:
samples = data.shape[0]
data = data[:,0]

plt.plot(data[:recordTime*samplerate]) # Plotting mono audio
plt.title("White Noise Test plot (Mono)")

In [ ]:
datafft = fft(data)
fftabs = abs(datafft)
freqs = fftfreq(samples,1/samplerate)
plt.xlim([10, samplerate/2])
plt.xscale( 'log' )
plt.grid( True )
plt.xlabel( 'Frequency (Hz)' )
plt.plot(freqs[:int(freqs.size/2)],fftabs[:int(freqs.size/2)])
plt.title("White Noise Test FFT Output plot")

### Burst Test

In [ ]:
# Running Burst Test & recording audio from RPI Zero W

frequency=2000
ncycles=5
interval=1
runTime=10

now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

st.burstTest(frequency,ncycles,interval)

In [ ]:
# Trimming the empty buffer time of wav file
#audio_input = ffmpeg.input(f'sineTest_{now}.wav')
audio_input = ffmpeg.input('burstTest_20210312_134736.wav')
audio_cut = audio_input.audio.filter('atrim', start=7, end=9)
#audio_output = ffmpeg.output(audio_cut, 'sineTest_{now}_trimmed.wav')
audio_output = ffmpeg.output(audio_cut, 'burstTest_20210312_134736_trimmed.wav')
ffmpeg.run(audio_output)

In [ ]:
# Loading in selected wav file data

# data, samplerate = soundfile.read(f"burstTest_{now}_trimmed.wav")
data, samplerate = soundfile.read("burstTest_20210312_134736_trimmed.wav")

In [ ]:
# Play recorded audio
ipd.Audio(f"burstTest_{now}_trimmed.wav") # load recent local sine test file

In [ ]:
#recordTime = 15
#samplerate = 48000
time=np.arange(0, runTime, 1/samplerate)

dataL = data[:,0]
dataR = data[:,1]


TOOLTIPS = [
    ("x", "$x"),
    ("y", "$y"),
]

# Create a new plot with title and axes labels
p = figure(title="Burst Test Stereo WAV", sizing_mode="stretch_width", tools='hover', tooltips=TOOLTIPS)


# change just some things about the x-axis
p.xaxis.axis_label = "Time (seconds)"
#p.xaxis.axis_label_text_color = ""
p.xaxis.axis_line_width = 3
p.xaxis.axis_line_color = "black"

# change just some things about the y-axis
p.yaxis.axis_label = "Normalised Min/Max Driver Voltage"
p.yaxis.major_label_text_color = "black"
p.yaxis.major_label_orientation = "vertical"

# Add a line renderer with legend and line thickness
p.line(time, dataL, legend_label="Left Channel Data", line_width=1, line_color="blue")
p.line(time, dataR, legend_label="Right Channel Data", line_width=1, line_color="red")

# Makes legends toggleable
p.legend.click_policy="hide"

# show the results
bop.output_notebook()
bop.show(p)

In [ ]:
samples = data.shape[0]
datafftL = fft(dataL)
datafftR = fft(dataR)
fftabsL = abs(datafftL[1:len(dataL)//2])
fftabsR = abs(datafftR[1:len(dataR)//2])
freqs = fftfreq(samples,1/(samplerate/2))



TOOLTIPS = [
    ("x", "$x"),
    ("y", "$y"),
]

# Create a new plot with title and axes labels
p = figure(title="Burst Test FFT", x_axis_type="log", x_range=(10,2*10**4), sizing_mode="stretch_width", tools='hover', tooltips=TOOLTIPS)

# change just some things about the x-axis
p.xaxis.axis_label = "Frequency (Hz)"
#p.xaxis.axis_label_text_color = ""
p.xaxis.axis_line_width = 3
p.xaxis.axis_line_color = "black"

# change just some things about the y-axis
p.yaxis.axis_label = "FFT Magnitude"
p.yaxis.major_label_text_color = "black"
p.yaxis.major_label_orientation = "vertical"

# Add a line renderer with legend and line thickness
p.line(freqs[:int(freqs.size/2)], fftabsL, legend_label="Left Channel Data FFT", line_width=1, line_color="blue")
p.line(freqs[:int(freqs.size/2)], fftabsR, legend_label="Right Channel Data FFT", line_width=1, line_color="red")

p.legend.click_policy="hide"

# show the results
bop.output_notebook()
bop.show(p)

In [ ]:
# data, samplerate = soundfile.read(f"burstTest_{now}.wav")
data, samplerate = soundfile.read("burstTest_20210312_134736.wav")

recordTime=15
samplerate=48000

plt.plot(data[:recordTime*samplerate]) # Plotting stereo audio
plt.title("Burst Test plot (Stereo)")

In [ ]:
samples = data.shape[0]
data = data[:,0]

plt.plot(data[:recordTime*samplerate]) # Plotting mono audio
plt.title("Burst Test plot (Mono)")

In [ ]:
datafft = fft(data)
fftabs = abs(datafft)
freqs = fftfreq(samples,1/samplerate)
plt.xlim([10, samplerate/2])
plt.xscale( 'log' )
plt.grid( True )
plt.xlabel( 'Frequency (Hz)' )
plt.plot(freqs[:int(freqs.size/2)],fftabs[:int(freqs.size/2)])
plt.title("Burst Test FFT Output plot")

### Ambient Noise Test

In [ ]:
# Running Ambient Noise Test & recording audio from RPI Zero W

now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

runTime = 5

st.ambientTest()

In [ ]:
data, samplerate = soundfile.read(f"Pi4ambientTest_{now}.wav")
#data, samplerate = soundfile.read("ambientTest_20210319_164908.wav")

In [ ]:
# Play recorded audio
#ipd.Audio(f"Pi4ambientTest_{now}.wav") # load recent local sine test file
ipd.Audio("ambientTest_20210319_164908.wav")

In [ ]:
recordTime = 15
samplerate = 48000
time=np.arange(0, 15, 1/samplerate)

dataL = data[:,0]
dataR = data[:,1]

# Set up figures
plot1 = figure(
    sizing_mode="stretch_width",
    title = "Ambient Noise Test Stereo WAV", 
    x_axis_label = 'Time (s)', 
    y_axis_label = 'Normalised Voltage',
    tools = 'xwheel_zoom,xpan,box_zoom,xwheel_pan,save,reset,hover'
    )

# Add a line renderer with legend and line thickness
plot1.line(time, dataL, legend_label="Left Channel Data", line_width=1, line_color="blue")
plot1.line(time, dataR, legend_label="Right Channel Data", line_width=1, line_color="red")

# Makes legends toggleable
plot1.legend.click_policy="hide"

# show the results
output_notebook()
show(plot1)

Socket exception: An existing connection was forcibly closed by the remote host (10054)


In [ ]:
samples = data.shape[0]
datafftL = fft(dataL)
datafftR = fft(dataR)
fftabsL = abs(datafftL[1:len(dataL)//2])
fftabsR = abs(datafftR[1:len(dataR)//2])
freqs = fftfreq(samples,1/(samplerate/2))

# Set up figures
plot1 = figure(
    sizing_mode="stretch_width",
    title = "Ambient Noise Test FFT", 
    x_axis_type="linear",
    y_axis_type="log",
    #x_range=(10,2*10**4),
    x_axis_label = 'Frequency (Hz)', 
    y_axis_label = 'Normalised Voltage',
    tools = 'xwheel_zoom,xpan,box_zoom,xwheel_pan,save,reset,hover'
    )

# Add a line renderer with legend and line thickness
plot1.line(freqs[1:int(freqs.size/2)], fftabsL, legend_label="Left Channel Data FFT", line_width=1, line_color="blue")
plot1.line(freqs[1:int(freqs.size/2)], fftabsR, legend_label="Right Channel Data FFT", line_width=1, line_color="red")

plot1.legend.click_policy="hide"

# show the results
output_notebook()
show(plot1)

###  <b> RPi Zero W recording </b>

In [18]:
data, samplerate = soundfile.read("2kHz5sStereo.wav")

In [19]:
ipd.Audio("2kHz5sStereo.wav")

In [ ]:
time=np.arange(0, 15, 1/samplerate)

dataL = data[:,0]
dataR = data[:,1]

# Set up figures
plot1 = figure(
    sizing_mode="stretch_width",
    title = "Old_Board WAV", 
    x_axis_label = 'Time (s)', 
    y_axis_label = 'Normalised Voltage',
    tools = 'xwheel_zoom,xpan,box_zoom,xwheel_pan,save,reset,hover'
    )

# Add a line renderer with legend and line thickness
plot1.line(time, dataL, legend_label="Left Channel Data", line_width=1, line_color="blue")
plot1.line(time, dataR, legend_label="Right Channel Data", line_width=1, line_color="red")

# Makes legends toggleable
plot1.legend.click_policy="hide"

# show the results
output_notebook()
show(plot1)

In [ ]:
samples = data.shape[0]
datafftL = fft(dataL)
datafftR = fft(dataR)
fftabsL = abs(datafftL[1:len(dataL)//2])
fftabsR = abs(datafftR[1:len(dataR)//2])
freqs = fftfreq(samples,1/(samplerate/2))

# Set up figures
plot1 = figure(
    sizing_mode="stretch_width",
    title = "Old Board FFT", 
    x_axis_type="linear",
    y_axis_type="log",
    #x_range=(10,2*10**4),
    x_axis_label = 'Frequency (Hz)', 
    y_axis_label = 'Normalised Voltage',
    tools = 'xwheel_zoom,xpan,box_zoom,xwheel_pan,save,reset,hover'
    )

# Add a line renderer with legend and line thickness
plot1.line(freqs[1:int(freqs.size/2)], fftabsL, legend_label="Left Channel Data FFT", line_width=1, line_color="blue")
plot1.line(freqs[1:int(freqs.size/2)], fftabsR, legend_label="Right Channel Data FFT", line_width=1, line_color="red")

plot1.legend.click_policy="hide"

# show the results
output_notebook()
show(plot1)